In [ ]:
!pip install beautifulsoup4 requests pandas numpy openpyxl openai python-dotenv selenium webdriver-manager httpx pydantic tqdm

In [ ]:
# Maximum pages to crawl per category
MAX_PAGE_PER_CATEGORY = 20

# Mapping of VN30 stock codes to their related keywords
KEYWORDS_MAP = {
    "ACB": ["ACB", "Ngân hàng ACB", "Ngân hàng TMCP Á Châu"],
    "BCM": ["BCM", "Becamex", "KCN Bình Dương", "khu công nghiệp Bình Dương", "VSIP", "Becamex IDC"],
    "BID": ["BIDV", "Ngân hàng Đầu tư và Phát triển Việt Nam"],
    "CTG": ["CTG", "VietinBank", "Ngân hàng Công Thương Việt Nam"],
    "DGC": ["DGC", "Hóa chất Đức Giang"],
    "FPT": ["FPT"],
    "GAS": ["PV GAS", "PV Gas", "Tổng Công ty Khí Việt Nam"],
    "GVR": ["GVR", "Tập đoàn Cao su", "Tập đoàn Công nghiệp Cao su Việt Nam"],
    "HDB": ["HDB", "HDBank", "Ngân hàng TMCP Phát triển Thành phố Hồ Chí Minh"],
    "HPG": ["HPG", "Hòa Phát"],
    "LPB": ["LPB", "LPBank", "LienVietPostBank", "Ngân hàng Bưu điện Liên Việt"],
    "MBB": ["MBB", "MBBank", "Ngân hàng Quân đội", "MB", "Ngân hàng TMCP Quân đội"],
    "MSN": ["MSN", "Masan", "WinCommerce"],
    "MWG": ["MWG", "Thế Giới Di Động", "Mobile World", "Bách Hóa Xanh", "BHX", "Điện Máy Xanh", "ĐMX", "TGDĐ"],
    "PLX": ["PLX", "Petrolimex", "Tập đoàn Xăng dầu Việt Nam"],
    "SAB": ["SAB", "Sabeco", "Tổng Công ty CP Bia - Rượu - Nước giải khát Sài Gòn"],
    "SHB": ["SHB", "Ngân hàng Thương mại Cổ phần Sài Gòn – Hà Nội", "Ngân hàng TMCP Sài Gòn Hà Nội"],
    "SSB": ["SSB", "Ngân hàng Thương mại Cổ phần Đông Nam Á", "Ngân hàng TMCP Đông Nam Á", "SeABank"],
    "SSI": ["SSI", "Chứng khoán SSI"],
    "STB": ["STB", "Sài Gòn Thương Tín", "Sacombank"],
    "TCB": ["TCB", "Techcombank", "Ngân hàng TMCP Kỹ Thương Việt Nam"],
    "TPB": ["TPB", "TPBank", "Ngân hàng Tiên Phong", "Ngân hàng TMCP Tiên Phong"],
    "VCB": ["VCB", "Vietcombank", "Ngân hàng TMCP Ngoại Thương Việt Nam", "Ngân hàng Ngoại thương"],
    "VHM": ["VHM", "Vinhomes"],
    "VIB": ["VIB", "Ngân hàng TMCP Quốc Tế Việt Nam", "Ngân hàng Quốc Tế"],
    "VIC": ["VIC", "Vingroup", "Công ty Cổ phần Tập đoàn Vingroup"],
    "VJC": ["VJC", "Vietjet Air", "Công ty Cổ phần Hàng không Vietjet", "máy bay Vietjet"],
    "VNM": ["VNM", "Vinamilk", "Công ty Cổ phần Sữa Việt Nam"],
    "VPB": ["VPB", "VPBank", "Ngân hàng TMCP Việt Nam Thịnh Vượng"],
    "VRE": ["VRE", "Vincom Retail", "Công ty Cổ phần Vincom Retail"]
}

def filter_articles_by_keywords(articles, keywords_map=KEYWORDS_MAP):
    # Filter articles containing VN30 stock keywords
    filtered = []
    
    for art in articles:
        # Convert title to lowercase for case-insensitive matching
        text = art["title"].lower()
        matched_codes = []
        
        # Check each stock code
        for code, keywords in keywords_map.items():
            for keyword in keywords:
                if keyword.lower() in text:
                    matched_codes.append(code)
                    break
        
        # Keep articles with at least one matched stock code
        if matched_codes:
            art_copy = art.copy()
            art_copy["codes"] = list(set(matched_codes))  # Remove duplicates
            filtered.append(art_copy)
    
    return filtered

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import json
from datetime import datetime

# HTTP headers to simulate a real browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'Origin': 'https://vietstock.vn',
    'Referer': 'https://vietstock.vn/tai-chinh.htm',
}

# Mapping of category URLs to their channel IDs
CATEGORY_CHANNEL_MAP = {
    'https://vietstock.vn/chung-khoan.htm': '144',
    'https://vietstock.vn/doanh-nghiep.htm': '733',
    'https://vietstock.vn/bat-dong-san.htm': '763',
    'https://vietstock.vn/tai-chinh.htm': '734',
    'https://vietstock.vn/hang-hoa.htm': '2',
    'https://vietstock.vn/kinh-te/vi-mo.htm': '761',
    'https://vietstock.vn/kinh-te/kinh-te-dau-tu.htm': '768',
    'https://vietstock.vn/the-gioi.htm': '736',
    'https://vietstock.vn/dong-duong.htm': '1317',
    'https://vietstock.vn/tai-chinh-ca-nhan.htm': '4259',
    'https://vietstock.vn/nhan-dinh-phan-tich.htm': '579',
    'https://vietstock.vn/san-giao-dich-tai-chinh.htm': '4645',
    'https://vietstock.vn/kinh-te.htm': '5307',
    'https://vietstock.vn/goc-nhin.htm': '4314',
}

def crawl_vietstock_category(url_category, channel_id, max_pages=3, output_file=None):
    # Update referer header for this category
    headers['Referer']=url_category
    session = requests.Session()
    articles_data = []
    
    # Step 1: Request the main page to get cookies
    response = session.get(url_category, headers=headers)
    
    if response.status_code != 200:
        return articles_data

    soup_main = BeautifulSoup(response.text, 'html.parser')
    
    # Get authentication token if available
    token = ""
    token_tag = soup_main.find('input', {'name': '__RequestVerificationToken'})
    if token_tag:
        token = token_tag.get('value')

    # Step 2: Loop through pages (pagination)
    api_url = "https://vietstock.vn/StartPage/ChannelContentPage"
    
    for page in range(1, max_pages + 1):
        # Prepare payload for API request
        payload = {
            'channelID': channel_id,
            'page': page,
            '__RequestVerificationToken': token
        }
        
        try:
            post_response = session.post(api_url, data=payload, headers=headers)
            
            if post_response.status_code == 200:
                soup_api = BeautifulSoup(post_response.text, 'html.parser')
                # Find all article titles
                articles = soup_api.find_all('h4', class_='channel-title') 
                
                if not articles:
                    articles = soup_api.select('h4 > a')

                # Extract article data
                for article in articles:
                    link_tag = article.find('a') if article.name != 'a' else article
                    
                    if link_tag:
                        title = link_tag.get('title') or link_tag.text.strip()
                        href = link_tag.get('href')
                        
                        # Build full URL if relative
                        if href and not href.startswith('http'):
                            full_link = f"https://vietstock.vn{href}"
                        else:
                            full_link = href
                        
                        # Create article data dictionary
                        article_data = {
                            'title': title,
                            'link': full_link,
                        }
                        articles_data.append(article_data)
                        
                        # Append to output file
                        if output_file:
                            with open(output_file, 'a', encoding='utf-8') as f:
                                f.write(json.dumps(article_data, ensure_ascii=False) + '\n')
            else:
                print(f"Lỗi khi gọi API trang {page}: {post_response.status_code}")
                
        except Exception as e:
            print(f"Có lỗi xảy ra: {e}")
            
        # Delay between requests
        time.sleep(2)
    
    return articles_data

# Output filename
output_filename = f'vietstock_articles.jsonl'

# Crawl all categories
all_articles = []
for category_url, channel_id in CATEGORY_CHANNEL_MAP.items():
    articles = crawl_vietstock_category(category_url, channel_id, max_pages=MAX_PAGE_PER_CATEGORY, output_file=output_filename)
    all_articles.extend(articles)
    time.sleep(3)  # Delay between categories

print(f"\n{'='*60}")
print(f"Hoàn thành! Đã lưu {len(all_articles)} bài báo vào file: {output_filename}")
print(f"{'='*60}")


Đang crawl category: https://vietstock.vn/chung-khoan.htm
Đang truy cập trang gốc: https://vietstock.vn/chung-khoan.htm...
Sử dụng Channel ID: 144

--- Đang cào trang 1 ---

--- Đang cào trang 1 ---
Tiêu đề: Giao dịch quỹ đầu tư: Lực mua “ẩn mình”
Link: https://vietstock.vn/2025/11/giao-dich-quy-dau-tu-luc-mua-an-minh-3358-1374635.htm
Tiêu đề: Phân tích kỹ thuật chứng khoán Việt Nam: Tuần 24-28/11/2025
Link: https://vietstock.vn/2025/11/phan-tich-ky-thuat-chung-khoan-viet-nam-tuan-24-28112025-585-1374582.htm
Tiêu đề: VIC dẫn dắt đà tăng, VN-Index có thêm gần 20 điểm trong tuần phân hóa
Link: https://vietstock.vn/2025/11/vic-dan-dat-da-tang-vn-index-co-them-gan-20-diem-trong-tuan-phan-hoa-830-1374585.htm
Tiêu đề: Chứng khoán phái sinh tuần 24-28/11/2025: Thanh khoản toàn thị trường liên tục giảm
Link: https://vietstock.vn/2025/11/chung-khoan-phai-sinh-tuan-24-28112025-thanh-khoan-toan-thi-truong-lien-tuc-giam-1636-1374572.htm
Tiêu đề: HOSE nhận hồ sơ niêm yết của Antesco
Link: https://

In [ ]:
# Read all articles from JSONL file
all_articles_list = []

with open(output_filename, 'r', encoding='utf-8') as f:
    for line in f:
        article = json.loads(line.strip())
        all_articles_list.append(article)

# Filter articles by VN30 keywords
filtered_articles = filter_articles_by_keywords(all_articles_list, keywords_map=KEYWORDS_MAP)

# Save filtered results to new file
filtered_filename = f'vietstock_articles_vn30_filtered.jsonl'

with open(filtered_filename, 'w', encoding='utf-8') as f:
    for article in filtered_articles:
        f.write(json.dumps(article, ensure_ascii=False) + '\n')

# Calculate statistics by stock code
code_stats = {}
for article in filtered_articles:
    for code in article['codes']:
        code_stats[code] = code_stats.get(code, 0) + 1

# Print statistics
print("\nThống kê số bài báo theo mã cổ phiếu:")
for code, count in sorted(code_stats.items(), key=lambda x: x[1], reverse=True):
    print(f"{code}: {count} bài")


Bắt đầu lọc bài báo theo keywords VN30...
Tổng số bài báo đã crawl: 280
Số bài báo khớp với VN30 keywords: 6

Đã lưu 6 bài báo VN30 vào file: vietstock_articles_vn30_filtered.jsonl

Thống kê số bài báo theo mã cổ phiếu:
MBB: 2 bài
VIC: 1 bài
MWG: 1 bài
VPB: 1 bài
SSI: 1 bài
